# Costruzione del database 
Questo database verrà utilizzato per creare gli algoritmi di Machine Learning. All'interno del database si trova il nome del paziente come indice di riga e il nome della variabile sulle colonne. Inoltre, sono stati calcolati alcuni parametri come:
* $\Delta t$ cioè la differenza di dose in $\mu Sv/h$ tra la curva di iniezione e controlaterale dopo 1min, 2min ... 6min
* $\Delta p$ cioè la differenza di dose in $\mu Sv/h$ fra il picco della curva di iniezione e il valore di dose medio del plateau


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
from sklearn import linear_model
from sklearn.metrics import r2_score
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter

In [ ]:
#files da elaborare
root_controlaterals = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/analisi_totale/contr"
root_injections = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/analisi_totale/inj"

In [ ]:
# Definire il nome delle acquisizioni e l'attività iniettata salvandoli in due array 
names = []
activities = []
for file_1 in os.listdir(root_injections):
    for file_2 in os.listdir(root_controlaterals):
        file_2 = file_2.split('_contr')[0] + '.csv'
        if file_1 == file_2:
            name, suffix = file_1.split('-')
            activity = int(suffix.split('MBq')[0])
            names.append(name)
            activities.append(activity)


In [ ]:
path_grafici = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/grafici"
database_con = pd.DataFrame()
database_inj = pd.DataFrame()
for n in names:
    for file_1 in os.listdir(root_injections):
        for file_2 in os.listdir(root_controlaterals):
            if n in file_1 and n in file_2:

                path_inj = os.path.join(root_injections, file_1)
                path_con = os.path.join(root_controlaterals, file_2)
                
                if n == '26_10_23_18F_10':
                    df_inj = pd.read_csv(path_inj,encoding = "ISO-8859-1", sep=';')
                else:
                    df_inj = pd.read_csv(path_inj, sep=';')
                    
                df_con = pd.read_csv(path_con, sep=';')
                
                control_inj = df_inj.columns[0]
                control_con = df_con.columns[0]
                
                if "line" in control_inj:
                    df_inj = pd.read_csv(path_inj, sep=';')
                    df_inj['Record time'] = pd.to_datetime(df_inj['Record time'])
                    df_inj.set_index('Record time', drop=True, inplace=True)
                    df_inj = df_inj.sort_index(ascending = True)
                    df_inj.index = pd.to_datetime(df_inj.index)
                    df_inj_Max = pd.to_numeric(df_inj['Dose Rate Max Value '])

                if "line" in control_con:
                    df_con = pd.read_csv(path_con, sep=';')
                    df_con['Record time'] = pd.to_datetime(df_con['Record time'])
                    df_con.set_index('Record time', drop=True, inplace=True)
                    df_con = df_con.sort_index(ascending = True)
                    df_con.index = pd.to_datetime(df_con.index)
                    df_con_Max = pd.to_numeric(df_con['Dose Rate Max Value '])


                    
                if "Filename" in control_inj:
                    df_inj = pd.read_csv(path_inj, sep=';', skiprows=lambda x: x in [0,2], header=1)
                    df_inj['Date / Time'] = pd.to_datetime(df_inj['Date / Time'])
                    df_inj.set_index('Date / Time', drop=True, inplace=True)
                    df_inj = df_inj.sort_index(ascending = True)
                    df_inj.index = pd.to_datetime(df_inj.index)
                    df_inj_ratemeter = df_inj[df_inj['Type']=='Ratemeter']
                    df_inj_Max = pd.to_numeric(df_inj_ratemeter['DR_Max'].apply(lambda x: x.replace(',', '.')))
                    

                if "Filename" in control_con:
                    df_con = pd.read_csv(path_con, sep=';', skiprows=lambda x: x in [0,2], header=1)
                    df_con['Date / Time'] = pd.to_datetime(df_con['Date / Time']) 
                    df_con.set_index('Date / Time', drop=True, inplace=True)
                    df_con = df_con.sort_index(ascending = True)
                    df_con.index = pd.to_datetime(df_con.index)
                    df_con_ratemeter = df_con[df_con['Type']=='Ratemeter']
                    df_con_Max = pd.to_numeric(df_con_ratemeter['DR_Max'].apply(lambda x: x.replace(',', '.')))


                    
                if "line" not in control_inj and "Filename" not in control_inj:
                    if n == '26_10_23_18F_10':
                        df_inj = pd.read_csv(path_inj,encoding = "ISO-8859-1", sep=';')
                        df_inj['Timestamp'] = pd.to_datetime(df_inj['Timestamp'])
                        df_inj.set_index('Timestamp', drop=True, inplace=True)
                        df_inj = df_inj.sort_index(ascending = True)
                        df_inj.index = pd.to_datetime(df_inj.index)
                        df_inj_Max = pd.to_numeric(df_inj['Intensità di dose (?Sv/h)'].apply(lambda x: x.replace(',', '.')))
                    else: 
                        df_inj = pd.read_csv(path_inj, sep=',')
                        df_inj['Timestamp'] = pd.to_datetime(df_inj['Timestamp'])
                        df_inj.set_index('Timestamp', drop=True, inplace=True)
                        df_inj = df_inj.sort_index(ascending = True)
                        df_inj.index = pd.to_datetime(df_inj.index)
                        df_inj_Max = pd.to_numeric(df_inj['Intensità di dose (μSv/h)'].apply(lambda x: x.replace(',', '.')))
                    
                if "line" not in control_con and "Filename" not in control_con:
                    df_con = pd.read_csv(path_con, sep=',')
                    df_con['Timestamp'] = pd.to_datetime(df_con['Timestamp'])
                    df_con.set_index('Timestamp', drop=True, inplace=True)
                    df_con = df_con.sort_index(ascending = True)
                    df_con.index = pd.to_datetime(df_con.index)
                    df_con_Max = pd.to_numeric(df_con['Intensità di dose (μSv/h)'].apply(lambda x: x.replace(',', '.')))
                    
                
                start_inj = df_inj_Max.index[0]
                start_con = df_con_Max.index[0]

                
                delta = np.absolute(start_inj-start_con)
                minutes_1 = delta.total_seconds() / 60
                
                if (float(minutes_1)>float(50)) and (start_inj > start_con):
                    df_inj_Max.index = df_inj_Max.index - timedelta(minutes=58.4)
                    
                    
                elif (float(minutes_1)>float(50)) and (start_inj < start_con):
                    df_con_Max.index = df_con_Max.index - timedelta(minutes=58.4) 
                    
                
                substring = "20_03"
                substring_1 = "23_03"

                    
                if substring in n or substring_1 in n:
                    if n!="20_03_18F_01" or n!="20_03_18F_05" or n!="20_03_18F_02":
                        df_con_Max.index = df_con_Max.index + timedelta(minutes=3) 
                    
                start_inj = df_inj_Max.index[0]
                start_con = df_con_Max.index[0]
                
                if start_inj < start_con:
                    df_con_1 = df_con_Max.loc[df_con_Max.index[0]:].reset_index(drop=True)
                    df_inj_1 = df_inj_Max.loc[df_con_Max.index[0]:].reset_index(drop=True)
                      
                elif start_inj > start_con:
                    df_con_1 = df_con_Max.loc[df_inj_Max.index[0]:].reset_index(drop=True)
                    df_inj_1 = df_inj_Max.loc[df_inj_Max.index[0]:].reset_index(drop=True)
                
                elif start_inj == start_con:
                    df_con_1 = df_con_Max.reset_index(drop=True)
                    df_inj_1 = df_inj_Max.reset_index(drop=True)
     
                
                if n == '22_09_18F_04':
                    diff = df_inj_1.diff()
                    maximum = diff.max()
                    maximum_time = diff.idxmax()
                    
                elif n == '26_10_23_18F_10':
                    #diff = df_inj.diff()
                    #maximum = df_inj.max()
                    maximum_time = df_inj_1.index[0]+60
                    print(maximum_time)
      
                else: 
                    maximum = df_inj_1.max()
                    maximum_time = df_inj_1.idxmax()
                    
                if maximum<900:
                    print(n)
                    print(maximum)
                    print(df_inj_1)
                    
                inj1 = df_inj_1.loc[maximum_time - 60: maximum_time + 540]
                con1 = df_con_1.loc[maximum_time - 60: maximum_time + 540]
                
                inj1.loc[180:] = gaussian_filter(inj1.loc[180:], sigma=5)
                con1.loc[180:] = gaussian_filter(con1.loc[180:], sigma=5)
                
               
                
                df_inj_2 = pd.DataFrame(inj1).reset_index(drop=True)
                df_con_2 = pd.DataFrame(con1).reset_index(drop=True)
            
                df_inj_2.columns= [n]
                df_con_2.columns= [n]


                database_con = pd.concat([database_con, df_con_2], axis=1)
                database_inj = pd.concat([database_inj, df_inj_2], axis=1)
                
#                 figure_name = n
                
#                 fig = plt.figure(figsize=(20,10))
#                 plt.ioff()
#                 plt.plot(database_inj[n],color='orange', linewidth= 3, label='Injection')
#                 plt.plot(database_con[n],color= 'blue',linestyle = 'dashed', linewidth= 3, label='Contralateral')
#                 plt.yscale('log')

#                 plt.xlabel("Time [s]", size = 30)
#                 plt.ylabel(r"Dose Rate [$\mu$ Sv/h]", size = 30)
#                 plt.xticks(fontsize=30)
#                 plt.yticks(fontsize=30)
#                 plt.xlim((0,600))
#                 plt.legend(fontsize=30)
#                 fig.savefig(path_grafici+'/'+ figure_name+'.png')
#                 plt.close()   

In [ ]:
# creazione riga del dataframe con tutte le attività
df_activity = pd.DataFrame(data = [activities], columns=names, index=['Activity (MBq)'])
df_activity

In [ ]:
#ci andiamo a calcolare tutti i valori delta t per ogni istante di tempo. In pratica delta t è definito come la differenza
#tra il valore di dose acquisito sul braccio di iniezione e il valore di dose acquisito sul braccio controlaterale
#allo stesso istante di tempo
subset_inj = database_inj.iloc[60:len(database_inj)]
subset_con = database_con.iloc[60:len(database_con)]
subset_delta = np.abs(subset_inj - subset_con).reset_index(drop=True)

In [ ]:
# Differenza tra delta t consecutivi, per vedere quanto decresce la differenza. Studiamo l'andamento delle differenze fra
#i valori di dose di iniezione e controlaterali
d = abs(subset_delta.diff())
d.drop(0,inplace = True)
d.reset_index(inplace = True)
d.drop('index',axis=1,inplace=True)
d

In [ ]:
print(d.head(0))

In [ ]:
#crea la tabella con i valori di delta t trovati sopra
#usiamo la differenza di dose allo stesso tempo tra iniezione e controlaterale
risultati = pd.DataFrame()
indice_iniziale = 60
for paziente in d.head(0):
    delta_dopo_plateau_60 = subset_delta[paziente].loc[indice_iniziale + 60] # / medie_timepoints[1]
    delta_dopo_plateau_120 = subset_delta[paziente].loc[indice_iniziale + 120] # / medie_timepoints[2]
    delta_dopo_plateau_180 = subset_delta[paziente].loc[indice_iniziale + 180] # / medie_timepoints[3]
    delta_dopo_plateau_240 = subset_delta[paziente].loc[indice_iniziale + 240] # / medie_timepoints[4]
    delta_dopo_plateau_300 = subset_delta[paziente].loc[indice_iniziale + 300] # / medie_timepoints[5]
    delta_dopo_plateau_360 = subset_delta[paziente].loc[indice_iniziale + 360] # / medie_timepoints[6]
    media_tempi = (delta_dopo_plateau_60+delta_dopo_plateau_120+delta_dopo_plateau_180+delta_dopo_plateau_240+delta_dopo_plateau_300+delta_dopo_plateau_360)/6
    info = [delta_dopo_plateau_60, delta_dopo_plateau_120, delta_dopo_plateau_180, delta_dopo_plateau_240, delta_dopo_plateau_300, delta_dopo_plateau_360,media_tempi]
    df = pd.DataFrame([info], index = [paziente], columns=['delta_dopo_1min', 'delta_dopo_2min' , 'delta_dopo_3min', 'delta_dopo_4min', 'delta_dopo_5min', 'delta_dopo_6min','media tempi'])
    risultati= pd.concat([risultati, df], axis=0)

In [ ]:
# Differenza tra delta t consecutivi, per vedere quanto decresce la differenza. Studiamo l'andamento delle differenze fra
#i valori di dose di iniezione e controlaterali
database_inj = database_inj.apply(pd.to_numeric)
d_inj = database_inj.iloc[60:len(database_inj)].reset_index(drop=True)
d_inj= np.abs(d_inj.diff())
d_inj.drop(0,inplace = True)
d_inj.reset_index(inplace = True)
d_inj.drop('index',axis=1,inplace=True)

In [ ]:
#picco, init_ind inizio del Plateau (calcolato prima)
#120+init_ind arriviamo al punto di inizio plateau fino a 6 minuti dopo e ci faccio la media. 
#Faccio una media dell'andamento 
df_parametri = pd.DataFrame()
indice_iniziale = 60
for paziente in d_inj.head(0):
    picco = database_inj[paziente].max()
    delta_p_60 = np.absolute(picco-database_inj[paziente].loc[60+indice_iniziale]) # / medie_timepoints[1]
    delta_p_120 = np.abs(picco-database_inj[paziente].loc[120+indice_iniziale]) # / medie_timepoints[2]
    delta_p_180 = np.abs(picco-database_inj[paziente].loc[180+indice_iniziale]) # / medie_timepoints[3]
    delta_p_240 = np.abs(picco-database_inj[paziente].loc[indice_iniziale + 240]) # / medie_timepoints[4]
    delta_p_300 = np.abs(picco-database_inj[paziente].loc[indice_iniziale + 300]) # / medie_timepoints[5]
    delta_p_360 = np.abs(picco-database_inj[paziente].loc[indice_iniziale + 360])
    media = np.mean(database_inj[paziente].iloc[60+indice_iniziale:60+indice_iniziale+300])
    
    info = [picco, np.absolute(picco-media),delta_p_60,delta_p_120,delta_p_180,delta_p_240,delta_p_300,delta_p_360] #delta p picco-media
    
    df = pd.DataFrame([info], index = [paziente], columns=['picco','differenza picco_plateau','dp_1min','dp_2min','dp_3min','dp_4min','dp_5min','dp_6min' ])
    df_parametri= pd.concat([df_parametri, df], axis=0)

In [ ]:
df_parametri

In [ ]:
df_activity_t = df_activity.transpose()
df_activity_t;

In [ ]:
df_totale = pd.concat([df_parametri, df_activity_t, risultati],axis=1)
df_tot_activity=pd.concat([df_parametri, df_activity_t, risultati],axis=1)
df_tot_picco=pd.concat([df_parametri, df_activity_t, risultati],axis=1)

In [ ]:
df_totale

In [ ]:
#normalizzazione rispetto all'attività
for i in df_tot_activity.columns:
    if i != 'picco'and i!='Activity (MBq)':
        df_tot_activity[i] = (df_tot_activity[i]/(df_tot_activity['Activity (MBq)']*df_tot_activity['picco']))*100
    if i=='Activity (MBq)':
        break

In [ ]:
df_tot_activity

In [ ]:
#normalizzazione rispetto all'attività
for i in df_tot_picco.columns:
    if i != 'picco'and i!='Activity (MBq)':
        df_tot_picco[i] = df_tot_picco[i]/df_tot_picco['picco']

In [ ]:
df_tot_picco

In [ ]:
df_totale.rename(columns = {0:'delta_p normalized'}, inplace = True)
#df_totale.drop('outcome', inplace=True, axis=1)

In [ ]:
df_tot_activity.to_excel('C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/nuova_normalizzazione_old.xlsx', sheet_name='sheet1', index=True)
#df_tot_picco.to_excel('C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/prova_norm_picco_new.xlsx', sheet_name='sheet1', index=True)

# Associare outcome a pazienti 

In [ ]:
file = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/nuova_normalizzazione_totale.xlsx"
outcome = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/outcome.xlsx"

data = pd.read_excel(file)
out = pd.read_excel(outcome)
names_data = data.iloc[:,0]
names_out = out.iloc[:,0]
lista=[]
nomi = []

for i in range(len(data)):
    for j in range(len(out)):

        if  names_out[j]==names_data[i]:
            lista.append(out.iloc[j,1])
            nomi.append(names_out[j])

In [ ]:
#controllo
for i in names_data:
    if i in nomi:
        print("ok")
    else:
        print(i)

In [ ]:
data.insert(2, "outcome", lista, True)

In [ ]:
data.to_excel('C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/nuova_norm_totale_output.xlsx', sheet_name='sheet1', index=True)

## Associare p e  t al SUV

In [ ]:
file = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Desktop/nuova_norm_totale_output.xlsx"
outcome = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/Tabella metriche stravaso_soglia10.xlsx"

In [ ]:
data = pd.read_excel(file)
out = pd.read_excel(outcome)

In [ ]:
names_data = data.iloc[:,1]
names_out = out.iloc[:,1]

In [ ]:
lista_p=[]
lista_t=[]
nomi = []
for i in range(len(data)):
    for j in range(len(out)):
        nomee = names_out[j].split("-")
        if  nomee[0]==names_data[i]:
            lista_p.append(data.iloc[j,4])
            lista_t.append(data.iloc[j,18])
            nomi.append(nomee[0])

In [ ]:
lista_p

In [ ]:
out.insert(15, "p", lista_p, True)

In [ ]:
out.to_excel('C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/prova_prova.xlsx', sheet_name='sheet1', index=True)

# TERAPIA

In [ ]:
#files da elaborare
root_controlaterals = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/Terapia/contr"
root_injections = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/Terapia/inj"

In [ ]:
# Definire il nome delle acquisizioni e l'attività iniettata salvandoli in due array 
names = []
activities = []
for file_1 in os.listdir(root_injections):
    for file_2 in os.listdir(root_controlaterals):
        file_2 = file_2.split('_contr')[0] + '.csv'
        if file_1 == file_2:
            name, suffix = file_1.split('-')
            activity = int(suffix.split('MBq')[0])
            names.append(name)
            activities.append(activity)


In [ ]:
path_grafici = "C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/Terapia/grafici"
database_con = pd.DataFrame()
database_inj = pd.DataFrame()
for n in names:
    for file_1 in os.listdir(root_injections):
        for file_2 in os.listdir(root_controlaterals):
            if n in file_1 and n in file_2:

                path_inj = os.path.join(root_injections, file_1)
                path_con = os.path.join(root_controlaterals, file_2)
                
                if n == '26_10_23_18F_10':
                    df_inj = pd.read_csv(path_inj,encoding = "ISO-8859-1", sep=';')
                else:
                    df_inj = pd.read_csv(path_inj, sep=';')
                    
                df_con = pd.read_csv(path_con, sep=';')
                
                control_inj = df_inj.columns[0]
                control_con = df_con.columns[0]
                
                if "line" in control_inj:
                    df_inj = pd.read_csv(path_inj, sep=';')
                    df_inj['Record time'] = pd.to_datetime(df_inj['Record time'])
                    df_inj.set_index('Record time', drop=True, inplace=True)
                    df_inj = df_inj.sort_index(ascending = True)
                    df_inj.index = pd.to_datetime(df_inj.index)
                    df_inj_Max = pd.to_numeric(df_inj['Dose Rate Max Value '])

                if "line" in control_con:
                    df_con = pd.read_csv(path_con, sep=';')
                    df_con['Record time'] = pd.to_datetime(df_con['Record time'])
                    df_con.set_index('Record time', drop=True, inplace=True)
                    df_con = df_con.sort_index(ascending = True)
                    df_con.index = pd.to_datetime(df_con.index)
                    df_con_Max = pd.to_numeric(df_con['Dose Rate Max Value '])


                    
                if "Filename" in control_inj:
                    df_inj = pd.read_csv(path_inj, sep=';', skiprows=lambda x: x in [0,2], header=1)
                    df_inj['Date / Time'] = pd.to_datetime(df_inj['Date / Time'])
                    df_inj.set_index('Date / Time', drop=True, inplace=True)
                    df_inj = df_inj.sort_index(ascending = True)
                    df_inj.index = pd.to_datetime(df_inj.index)
                    df_inj_ratemeter = df_inj[df_inj['Type']=='Ratemeter']
                    df_inj_Max = pd.to_numeric(df_inj_ratemeter['DR_Max'].apply(lambda x: x.replace(',', '.')))
                    

                if "Filename" in control_con:
                    df_con = pd.read_csv(path_con, sep=';', skiprows=lambda x: x in [0,2], header=1)
                    df_con['Date / Time'] = pd.to_datetime(df_con['Date / Time']) 
                    df_con.set_index('Date / Time', drop=True, inplace=True)
                    df_con = df_con.sort_index(ascending = True)
                    df_con.index = pd.to_datetime(df_con.index)
                    df_con_ratemeter = df_con[df_con['Type']=='Ratemeter']
                    df_con_Max = pd.to_numeric(df_con_ratemeter['DR_Max'].apply(lambda x: x.replace(',', '.')))


                    
                if "line" not in control_inj and "Filename" not in control_inj:
                    if n == '26_10_23_18F_10':
                        df_inj = pd.read_csv(path_inj,encoding = "ISO-8859-1", sep=';')
                        df_inj['Timestamp'] = pd.to_datetime(df_inj['Timestamp'])
                        df_inj.set_index('Timestamp', drop=True, inplace=True)
                        df_inj = df_inj.sort_index(ascending = True)
                        df_inj.index = pd.to_datetime(df_inj.index)
                        df_inj_Max = pd.to_numeric(df_inj['Intensità di dose (?Sv/h)'].apply(lambda x: x.replace(',', '.')))
                    else: 
                        df_inj = pd.read_csv(path_inj, sep=',')
                        df_inj['Timestamp'] = pd.to_datetime(df_inj['Timestamp'])
                        df_inj.set_index('Timestamp', drop=True, inplace=True)
                        df_inj = df_inj.sort_index(ascending = True)
                        df_inj.index = pd.to_datetime(df_inj.index)
                        df_inj_Max = pd.to_numeric(df_inj['Intensità di dose (μSv/h)'].apply(lambda x: x.replace(',', '.')))
                    
                if "line" not in control_con and "Filename" not in control_con:
                    df_con = pd.read_csv(path_con, sep=',')
                    df_con['Timestamp'] = pd.to_datetime(df_con['Timestamp'])
                    df_con.set_index('Timestamp', drop=True, inplace=True)
                    df_con = df_con.sort_index(ascending = True)
                    df_con.index = pd.to_datetime(df_con.index)
                    df_con_Max = pd.to_numeric(df_con['Intensità di dose (μSv/h)'].apply(lambda x: x.replace(',', '.')))
                    
                
                start_inj = df_inj_Max.index[0]
                start_con = df_con_Max.index[0]

                
                delta = np.absolute(start_inj-start_con)
                minutes_1 = delta.total_seconds() / 60
                
                if (float(minutes_1)>float(50)) and (start_inj > start_con):
                    df_inj_Max.index = df_inj_Max.index - timedelta(minutes=58.4)
                    
                    
                elif (float(minutes_1)>float(50)) and (start_inj < start_con):
                    df_con_Max.index = df_con_Max.index - timedelta(minutes=58.4) 
                    
                
                substring = "20_03"
                substring_1 = "23_03"

                    
                if substring in n or substring_1 in n:
                    if n!="20_03_18F_01" or n!="20_03_18F_05" or n!="20_03_18F_02":
                        df_con_Max.index = df_con_Max.index + timedelta(minutes=3) 
                    
                start_inj = df_inj_Max.index[0]
                start_con = df_con_Max.index[0]
                
                if start_inj < start_con:
                    df_con_1 = df_con_Max.loc[df_con_Max.index[0]:].reset_index(drop=True)
                    df_inj_1 = df_inj_Max.loc[df_con_Max.index[0]:].reset_index(drop=True)
                      
                elif start_inj > start_con:
                    df_con_1 = df_con_Max.loc[df_inj_Max.index[0]:].reset_index(drop=True)
                    df_inj_1 = df_inj_Max.loc[df_inj_Max.index[0]:].reset_index(drop=True)
                
                elif start_inj == start_con:
                    df_con_1 = df_con_Max.reset_index(drop=True)
                    df_inj_1 = df_inj_Max.reset_index(drop=True)
     
                
                
                #diff = df_inj_1.diff()
                #maximum = diff.max()
                #maximum_time = diff.idxmax()
                
                maximum_time = df_inj_1.index[0]+120
      
                if n=="09_10_177Lu_01" or n=="30_01_177Lu_01":
                    diff = df_inj_1.diff()
                    maximum = diff.max()
                    maximum_time = diff.idxmax()
  
                    
                inj1 = df_inj_1.loc[maximum_time - 120: maximum_time + 900]
                con1 = df_con_1.loc[maximum_time - 120: maximum_time + 900]
                
                inj1 = gaussian_filter(inj1, sigma=5)
                con1 = gaussian_filter(con1, sigma=5)
                
               
                
                df_inj_2 = pd.DataFrame(inj1).reset_index(drop=True)
                df_con_2 = pd.DataFrame(con1).reset_index(drop=True)
            
                df_inj_2.columns= [n]
                df_con_2.columns= [n]


                database_con = pd.concat([database_con, df_con_2], axis=1)
                database_inj = pd.concat([database_inj, df_inj_2], axis=1)
                
#                 figure_name = n
                
#                 fig = plt.figure(figsize=(20,10))
#                 plt.ioff()
#                 plt.plot(database_inj[n],color='orange', linewidth= 3, label='Injection')
#                 plt.plot(database_con[n],color= 'blue',linestyle = 'dashed', linewidth= 3, label='Contralateral')
#                 plt.yscale('log')

#                 plt.xlabel("Time [s]", size = 30)
#                 plt.ylabel(r"Dose Rate [$\mu$ Sv/h]", size = 30)
#                 plt.xticks(fontsize=30)
#                 plt.yticks(fontsize=30)
#                 plt.xlim((0,960))
#                 plt.legend(fontsize=30)
#                 fig.savefig(path_grafici+'/'+ figure_name+'.png')
#                 plt.close()   

In [ ]:
# creazione riga del dataframe con tutte le attività
df_activity = pd.DataFrame(data = [activities], columns=names, index=['Activity (MBq)'])
df_activity

In [ ]:
#ci andiamo a calcolare tutti i valori delta t per ogni istante di tempo. In pratica delta t è definito come la differenza
#tra il valore di dose acquisito sul braccio di iniezione e il valore di dose acquisito sul braccio controlaterale
#allo stesso istante di tempo
subset_inj = database_inj.iloc[60:len(database_inj)]
subset_con = database_con.iloc[60:len(database_con)]
subset_delta = np.abs(subset_inj - subset_con).reset_index(drop=True)

In [ ]:
# Differenza tra delta t consecutivi, per vedere quanto decresce la differenza. Studiamo l'andamento delle differenze fra
#i valori di dose di iniezione e controlaterali
d = abs(subset_delta.diff())
d.drop(0,inplace = True)
d.reset_index(inplace = True)
d.drop('index',axis=1,inplace=True)
d

In [ ]:
#crea la tabella con i valori di delta t trovati sopra
#usiamo la differenza di dose allo stesso tempo tra iniezione e controlaterale
risultati = pd.DataFrame()
indice_iniziale = 60
for paziente in d.head(0):
    delta_dopo_plateau_60 = subset_delta[paziente].loc[indice_iniziale + 60] # / medie_timepoints[1]
    delta_dopo_plateau_120 = subset_delta[paziente].loc[indice_iniziale + 120] # / medie_timepoints[2]
    delta_dopo_plateau_180 = subset_delta[paziente].loc[indice_iniziale + 180] # / medie_timepoints[3]
    delta_dopo_plateau_240 = subset_delta[paziente].loc[indice_iniziale + 240] # / medie_timepoints[4]
    delta_dopo_plateau_300 = subset_delta[paziente].loc[indice_iniziale + 300] # / medie_timepoints[5]
    delta_dopo_plateau_360 = subset_delta[paziente].loc[indice_iniziale + 360] # / medie_timepoints[6]
    delta_dopo_plateau_480 = subset_delta[paziente].loc[indice_iniziale + 480] # / medie_timepoints[6]
    delta_dopo_plateau_600 = subset_delta[paziente].loc[indice_iniziale + 600] # / medie_timepoints[6]
    media_tempi = (delta_dopo_plateau_60+delta_dopo_plateau_120+delta_dopo_plateau_180+delta_dopo_plateau_240+delta_dopo_plateau_300+delta_dopo_plateau_360+delta_dopo_plateau_480+delta_dopo_plateau_600)/8
    info = [delta_dopo_plateau_60, delta_dopo_plateau_120, delta_dopo_plateau_180, delta_dopo_plateau_240, delta_dopo_plateau_300, delta_dopo_plateau_360,delta_dopo_plateau_480,delta_dopo_plateau_600,media_tempi]
    df = pd.DataFrame([info], index = [paziente], columns=['delta_dopo_1min', 'delta_dopo_2min' , 'delta_dopo_3min', 'delta_dopo_4min', 'delta_dopo_5min', 'delta_dopo_6min','delta_dopo_8min','delta_dopo_10min','media tempi'])
    risultati= pd.concat([risultati, df], axis=0)

In [ ]:
risultati

In [ ]:
risultati.to_excel("C:/Users/39333/OneDrive - Università degli Studi di Padova/Documenti/Progetto Specialistica/Nuovi/acquisizioni 07 e 11-2023/risultati_terapia.xlsx",sheet_name='sheet1', index=True)